In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math

In [41]:
df = pd.read_csv('Data laboratorio 1/students.csv', sep=';')
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [42]:
df['Target'].unique()

array(['Dropout', 'Graduate', 'Enrolled'], dtype=object)

In [43]:
#Unificando Graduate + Enrolled
df['Target']=df['Target'].replace('Graduate','Enrolled')
df['Target'].unique()

array(['Dropout', 'Enrolled'], dtype=object)

In [44]:
#Separando en train test



In [45]:
#Tipos de datos, buscando los datos continuos
pd.set_option('display.max_rows', None)
df.dtypes

Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance\t                        int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender        

In [46]:
print(df.select_dtypes(include=['float']).columns)

Index(['Previous qualification (grade)', 'Admission grade',
       'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)',
       'Unemployment rate', 'Inflation rate', 'GDP'],
      dtype='object')


#### Preprocesing
* Unificar los valores graduate enrolled de Target
* Pasar los datos continuos a low mid y high (según aplique)
    * Previous Qualification
    * Admission Grade
    * Curricular units 1st sem (grade) - (no dice continuous en la pagina pero float)
    * Curricular units 2nd sem (grade) -    "
    * Unemployment rate
    * Inflation rate
    * GDP


In [47]:
#Pasando a categorías los calificaciones previas
df['Previous qualification (grade)'] = pd.cut(df['Previous qualification (grade)'], bins=3, labels=["low",'mid','high'], right=False)


In [48]:
df['Previous qualification (grade)'].unique()

['low', 'high', 'mid']
Categories (3, object): ['low' < 'mid' < 'high']

In [49]:
columns = df.select_dtypes(include=['float']).columns
for column in columns:
    df[column] = pd.cut(df[column], bins=3, labels=["low",'mid','high'], right=False)


In [50]:
df[df["Target"]=="Enrolled"].shape[0]

3003

#### ID3

In [51]:
#Entropía total
entropia(df, "Target", df["Target"].unique())

0.6277725714164744

In [52]:
#Calculo de la entropía total con respecto a la columna objetio
def entropia(df, objetivo, resultados):
    entropia = [0] * len(resultados)
    for i, val in enumerate(resultados):
        ci = df[df[objetivo]==val].shape[0]
        tot = df.shape[0]
        entropia[i]= -ci/tot * math.log(ci/tot) if not (ci==0) else 0
    return sum(entropia)

In [53]:
#Entropía del subconjunto
def entropiasbcjto(df, atributo, objetivo): #df datos, atributo que separa, columna objetivo
    valores = df[atributo].unique() #valores posibles del atributo
    aux = [0]*len(valores)
    s = df.shape[0] # cantidad de datos
    for i,valor in enumerate(valores):
        sv = df[df[atributo]==valor].shape[0] #cantidad de datos con atributo = valor
        EntrSv = entropia(df[df[atributo]==valor], objetivo, df[objetivo].unique())#Entropía de cada valor posible del atributo
        aux[i]=sv/s*EntrSv
    return sum(aux)

In [54]:
entropiasbcjto(df,"Application mode", "Target")

0.5833968508234656

In [55]:
# Information Gain
def info_gain(df, atributo, objetivo):
    return entropia(df, objetivo, df[objetivo].unique()) - entropiasbcjto(df, atributo, objetivo)

In [56]:
info_gain(df,"Gender", "Target")

0.020410823899658648

In [59]:
def max_info (df, objetivo):
    atributos = df.drop(objetivo,axis = 1).columns
    max_info = -1
    max_info_atrib = None

    for col in atributos:
        gain_col = info_gain(df, col, objetivo)

        if gain_col > max_info:
            max_info = gain_col
            max_info_atrib = col
    
    return max_info_atrib



In [60]:
max_info(df, "Target")

'Curricular units 2nd sem (approved)'

In [61]:
max_info(df.drop('Curricular units 2nd sem (approved)',axis=1), "Target")

'Curricular units 1st sem (approved)'

In [69]:
valores_cantidades = df["Nacionality"].value_counts(sort=False)
for valor, cantidad in valores_cantidades.items():
    print(valor,cantidad)
print

1 4314
62 2
6 13
41 38
26 14
103 3
13 1
25 2
21 2
101 2
11 3
22 13
32 1
100 3
24 5
109 1
2 2
108 1
105 2
14 1
17 1


<function print>

In [109]:
resultados = df["Target"].unique()
print(resultados)
len(df[df["Target"]==resultados[0]])

['Dropout' 'Enrolled']


1421

In [113]:
#Generar sub arbol a partir de un atributo
def generar_sub_arbol(atributo, data, objetivo, min_gain_split):
    resultados = data[objetivo].unique()
    valores = data[atributo].unique()
    tree = {} #subarbol

    for valor in valores:
        datos_valor = data[data[atributo]==valor] # me quedo con los datos de esta rama
        sig_nodo=max_info(datos_valor,objetivo) 
        if info_gain(datos_valor, sig_nodo, objetivo) <= min_gain_split:#si no supera el umbaral de ganancia
            #cantidad de datos de cada etiqueta
            cant0 = len(datos_valor[datos_valor[objetivo]==resultados[0]])
            cant1 = len(datos_valor[datos_valor[objetivo]==resultados[1]])
            tree[valor] = resultados[0] if cant0>cant1 else resultados[1] #asigno la etiqueta mayoritaria
            
        
        if info_gain(datos_valor, sig_nodo , objetivo) > min_gain_split: #Si la rama aporta información separo
            tree[valor]="?" #marco la rama a extender
        
        data = data[data[atributo]!=valor] #dejo todos los datos de esta rama
    return tree, data

In [114]:
tree, data = generar_sub_arbol("Nacionality",df,"Target",0)


In [115]:
tree

{1: '?',
 62: 'Enrolled',
 6: '?',
 41: '?',
 26: '?',
 103: '?',
 13: 'Enrolled',
 25: 'Enrolled',
 21: '?',
 101: '?',
 11: 'Enrolled',
 22: '?',
 32: 'Enrolled',
 100: '?',
 24: '?',
 109: 'Dropout',
 2: 'Enrolled',
 108: 'Enrolled',
 105: '?',
 14: 'Enrolled',
 17: 'Dropout'}

In [ ]:
#Generar árbol

def generar_arbol(padre, data, objetivo):
    if padre == None: # si es el comienzo del arbol
        padre, data = generar_sub_arbol(max_info(data,objetivo),)